In [148]:
from pyxing.session import *
from pyxing.query import *

import pandas as pd
import numpy as np

import threading

import time
from datetime import datetime

import pythoncom
import win32com.client as winAPI

In [149]:
shcode_list = ['A033780', 'A064350', 'A017670', 'A005490', 'A014680', 'A012450', 'A051900', 'A009150', 'A066570', 'A108320', 'A047810', 'A000660', 'A010140', 'A035420', 'A005380', 'A009830', 'A052690', 'A034020', 'A005070', 'A005420', 'A042700', 'A028050', 'A068270', 'A137310', 'A005930', 'A017960', 'A075580', 'A003490']

In [150]:
for shcode in shcode_list:
    globals()['df_{}'.format(shcode)] = pd.DataFrame(columns=['종목코드','시간','현재가','체결수량', '거래량', '매도체결수량', '매수체결수량'])

In [151]:
receive_list = ['shcode','chetime','price','cvolume','volume','mdvolume','msvolume']

In [152]:
import win32com.client
import pythoncom

class XingRealTime(object):
    realTime = None

    def __init__(self):
        self.realTime = win32com.client.DispatchWithEvents("XA_DataSet.XAReal", RealTimeDataHandler)
        RealTimeDataHandler.realTime = self.realTime
    #def end

    

    def subscribeRealTimeData(self, shcode_list):
        # 실시간 데이터 요청 
        self.realTime.LoadFromResFile("C:\\eBEST\\xingAPI\\Res\\S3_.res")
        self.realTime.SetFieldData('InBlock', 'shcode', shcode_list)
        self.realTime.AdviseRealData()
    #def end

    def add_item(self, shcode):
    # 실시간데이터 요청 종목 추가
        self.SetFieldData("InBlock", "shcode", shcode)
        self.AdviseRealData()

    def waitEvent(self):
        # Waiting query event
        while RealTimeDataHandler.realTimeState == 0:
            pythoncom.PumpWaitingMessages()
    #def end


# 전송시간에 따른 시간 정보 가져오기
def recieve_dt(self,shcode,i):
    xaquery = XAQuery()
    re_df = self.realTime.ReceivedRealData('S3_',shcode=shcode)
    re_time = xaquery.set_field_data('t0167',id=re_df[0]['chetime'])


    dttime = (re_time[0]['dt']+re_time[0]['time']).values[0]
    re_df = re_df[0][receive_list[1:]].values[0].tolist()

    df = globals()['df_{}'.format(shcode)]

    df.loc[i, 'Datetime'] = dttime
    df.iloc[i,1:] = re_df
#class end


class RealTimeDataHandler:
    realTimeState = 0
    realTime = None

    def OnReceiveRealData(self,XingRealTime):
        shcode = self.realTime.GetFieldData('OutBlock', 'shcode')
        chetime = self.realTime.GetFieldData('OutBlock', 'chetime')
        price = self.realTime.GetFieldData('OutBlock', 'price')
        cvolume = self.realTime.GetFieldData('OutBlock', 'cvolume')
        volume = self.realTime.GetFieldData('OutBlock', 'cvolume')
        mdvolume = self.realTime.GetFieldData('OutBlock', 'mdvolume')
        msvolume = self.realTime.GetFieldData('OutBlock', 'msvolume')
        print("종목;{0},시간;{1}, 현재가;{2}, 체결수량;{3}, 거래량;{4}, 매도누적체결수량;{5}, 매수누적체결수량;{6}".format(shcode, chetime, price, cvolume, volume, mdvolume, msvolume))
        
    #def end
# class end

    @classmethod
    def get_instance(cls):
        xreal = win32com.client.DispatchWithEvents("XA_DataSet.XAReal", cls)
        return xreal

In [153]:
def multithreading_xing(i):
    start = time.perf_counter()
    threads=[]

    for shcode in shcode_list:
        t = threading.Thread(target=recieve_dt(shcode,i))
        t.start()
        threads.append(t)

    for thread in threads:
        thread.join()

    finish = time.perf_counter()


In [154]:
datetime.now().strftime('%M')

'40'

In [155]:
i = 0
while True:
    multithreading_xing(i)
    
    time.sleep(60)
    
    i+=1

    # 매시 정각 마다 출력 
    if datetime.now().strftime('%M') == '00': 
        print(i)


    # 15시30분 수집 종료
    if datetime.now().strftime('%H:%M') == '15:30':
        # print(i)
        break

TypeError: recieve_dt() missing 1 required positional argument: 'i'